In [ ]:
# Importing the libraries
%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
import seaborn as sns; sns.set()
import tensorflow as tf

In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

In [ ]:
# Reading the dataset
data = pd.read_csv('./data/fashion/fashion-mnist_train.csv')

In [ ]:
data.tail()

In [ ]:
# Extracting label column from the data
labels = data['label']
data = data.drop(columns=['label'])

In [ ]:
data.shape

In [ ]:
# We should use axis-0 for substracting the mean on PCA
data.mean(axis=0).shape

In [ ]:
# Normalizing the pixels in range [0.-1.]
data = data / 255.

### PCA on scikit-learn library (Baseline)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
projected = pca.fit_transform(data)

In [ ]:
# Plotting the projected data
plt.scatter(projected[:, 0], projected[:, 1],
            c=labels, edgecolor='none', alpha=0.9,
            cmap=plt.cm.get_cmap('plasma', 10))
plt.xlabel('first principle component')
plt.ylabel('second principle component')
plt.colorbar();

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
projected = pca.fit_transform(data)

In [ ]:
# Plotting the projected data
plt.scatter(projected[:, 0], projected[:, 1], projected[:, 2],
            c=labels, edgecolor='none', alpha=0.9,
            cmap=plt.cm.get_cmap('plasma', 10))
plt.xlabel('first principle component')
plt.ylabel('second principle component')
plt.colorbar();

### My PCA implementation

In [ ]:
def my_PCA(data, num_components=2):   
    data -= data.mean(axis=0)
    C = np.cov(data, rowvar=False)
    eig_vals, eig_vecs = linalg.eigh(C)
    
    idx = np.argsort(eig_vals)[::-1]
    eig_vecs = eig_vecs[:, idx]
    eig_vals = eig_vals[idx]
    
    eig_vecs = eig_vecs[:, :num_components]
    
    projection = np.dot(eig_vecs.T, data.T).T
    
    return projection

In [ ]:
projected = my_PCA(data)

In [ ]:
plt.scatter(projected[:, 0], projected[:, 1],
            c=labels, edgecolor='none', alpha=0.9,
            cmap=plt.cm.get_cmap('plasma', 10))
plt.xlabel('first principle component')
plt.ylabel('second principle component')
plt.colorbar();

In [ ]:
projected = my_PCA(data, num_components=3)

In [ ]:
plt.scatter(projected[:, 0], projected[:, 1], projected[:, 2],
            c=labels, edgecolor='none', alpha=0.9,
            cmap=plt.cm.get_cmap('plasma', 10))
plt.xlabel('first principle component')
plt.ylabel('second principle component')
plt.colorbar();

### My neural network implementation

In [ ]:
# Defining placeholder and variables in the scope of GPU
with tf.device("/gpu:0"):
    x = tf.placeholder(shape=[None, 784], dtype=tf.float32)

    w1 = tf.Variable(tf.truncated_normal(stddev=.1, shape=[784, 256]), 
                     dtype=tf.float32)
    w2 = tf.Variable(tf.truncated_normal(stddev=.1, shape=[256, 64]), 
                     dtype=tf.float32)
    w3 = tf.Variable(tf.truncated_normal(stddev=.1, shape=[64, 2]), 
                     dtype=tf.float32)
    
    # neural network layers
    hidden_1 = tf.nn.relu(tf.matmul(x, w1))
    hidden_2 = tf.nn.relu(tf.matmul(hidden_1, w2))
    projection = tf.matmul(hidden_2, w3)
    hidden_4 = tf.nn.relu(projection)
    hidden_5 = tf.nn.relu(tf.matmul(hidden_4, tf.transpose(w3)))
    hidden_6 = tf.nn.relu(tf.matmul(hidden_5, tf.transpose(w2)))
    y = tf.matmul(hidden_6, tf.transpose(w1))
    
    # loss function and optimizer
    loss = tf.reduce_mean(tf.reduce_sum((x - y) * (x - y), 1))
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    init = tf.initialize_all_variables()

In [ ]:
with tf.Session() as session:
    session.run(init)
    feed_dict = {x: data}

    for batch in range(100000):
        session.run(optimizer, feed_dict=feed_dict)

        loss_, proj_ = session.run([loss, projection],
                                         feed_dict=feed_dict)
        
        if batch % 100 == 0:
            plt.clf()
            plt.scatter(proj_[:, 0], proj_[:, 1],
                        c=labels, edgecolor='none', alpha=0.9,
                        cmap=plt.cm.get_cmap('plasma', 10))
            plt.title('batch %d, loss %g' % (batch, loss_))
            plt.show(False)
            plt.pause(0.001)

In [ ]:
# Defining placeholder and variables in the scope of GPU
with tf.device("/gpu:0"):
    x = tf.placeholder(shape=[None, 784], dtype=tf.float32)

    w1 = tf.Variable(tf.truncated_normal(stddev=.1, shape=[784, 256]), 
                     dtype=tf.float32)
    w2 = tf.Variable(tf.truncated_normal(stddev=.1, shape=[256, 64]), 
                     dtype=tf.float32)
    w3 = tf.Variable(tf.truncated_normal(stddev=.1, shape=[64, 3]), 
                     dtype=tf.float32)
    
    # neural network layers
    hidden_1 = tf.nn.relu(tf.matmul(x, w1))
    hidden_2 = tf.nn.relu(tf.matmul(hidden_1, w2))
    projection = tf.matmul(hidden_2, w3)
    hidden_4 = tf.nn.relu(projection)
    hidden_5 = tf.nn.relu(tf.matmul(hidden_4, tf.transpose(w3)))
    hidden_6 = tf.nn.relu(tf.matmul(hidden_5, tf.transpose(w2)))
    y = tf.matmul(hidden_6, tf.transpose(w1))
    
    # loss function and optimizer
    loss = tf.reduce_mean(tf.reduce_sum((x - y) * (x - y), 1))
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    init = tf.initialize_all_variables()

In [ ]:
%matplotlib inline
with tf.Session() as session:
    session.run(init)
    feed_dict = {x: data}

    for batch in range(100000):
        session.run(optimizer, feed_dict=feed_dict)

        loss_, proj_ = session.run([loss, projection],
                                         feed_dict=feed_dict)
        
        if batch % 100 == 0:
            plt.clf()
            plt.scatter(proj_[:, 0], proj_[:, 1], proj_[:, 2],
                        c=labels, edgecolor='none', alpha=0.9,
                        cmap=plt.cm.get_cmap('plasma', 10))
            plt.title('batch %d, loss %g' % (batch, loss_))
            plt.show(False)
            plt.pause(0.001)